# Synopsis

Unstructured text is one of the most plentiful sources of data in many disciplines. However, because this data is unstructured (meaning that it isn't organized nicely into an excel spreadsheet) even basic analysis can be a bit more involved than with other data. In this unit we will go over the basics of textual analysis and cover:

* Techniques for **parsing** large-scale text
* Basic **bag of words** analysis
* Examining **distributions** of word usage

# Read libraries

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from colorama import Back, Fore, Style
from pathlib import Path
from sys import path

path.append('../My_libraries')
path

In [ ]:
my_fontsize = 15

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

from collections import Counter
from random import random
from string import punctuation, whitespace

from My_libraries.my_stats import half_frame

# Text as data

Whether it's extracting numerical data from text, or dealing with text directly, the ability to manipulate text in the form of strings is essential for any number of data science projects. More importantly, analyzing text allows for quantitative analysis in a number of areas that would be prohibitive otherwise.

Over a decade ago, Google convinced librarians around the world to scan their books for the [Google Books Project](https://www.newyorker.com/business/currency/what-ever-happened-to-google-books). (At the time, people were still buying into their *Do no harm* bullshit $-$ the [Wikipedia page](https://en.wikipedia.org/wiki/Google_Books) provides a less wide-eyed version).  

While Google intentions were far from altruistic, the scanning and digitization ended up creating an exciting resource that has enabled all sorts of previously impossible analyses. For example [Michel et al.](https://www.science.org/doi/10.1126/science.1199644) studied the changes in occurrences of specific words or groups of words in books over time:

<img src = "Images/electronic_book_use.png" width = 600>


Quantitative textual analysis makes the task of longitudinal analysis or simply large-scale text possible. However, one important point that I want to impress upon you is that **textual analysis is not nearly as simple as it seems**.  Teaching computers how to perform tasks that are innately easy to you can be quite challenging. 


# The Data

While the Google's motivation was problematic, a competing effort $-$ [Project Gutenberg](http://www.gutenberg.org) $-$ was digitizing thousands of post-copyright books and making them freely available to all in numerous formats.

<table>
    <tr>
        <td>
            <img src = 'Images/qPSwl88-_400x400.png' width = 80>
        </td>
        <td>
            <img src = 'Images/gutenberg_twain.png' width = 500>
        </td>
    </tr>
</table>


Luckily for us, this includes the complete works of William Shakespeare which we've pre-downloaded for you.

<img src = 'Images/gutenberg_shakespeare.png' width = 600>


For data processing in a computer, the best format is typically **plain text**.



First things first, **ALWAYS LOOK AT YOUR DATA**. 

<img src = 'Images/complete_works.png' width = 600>



Now is the time to open [Shakespeare.txt](Data/Shakespeare.txt).

**So, what questions do you want to be able to answer in order to start working?**

> 1. How are different plays separated from one another?
>
> 2. How is dialogue formatted?
>
> 3. What extraneous information might we want to ignore?
>
> 4. How "well behaved" is our dataset? (i.e. is the formatting general or unique for different plays?)

We will need the answers to all those questions in order to be able to write code to analyze the text. 

<br>

<br>

<br>

<br>


But, first, let's read the file...


In [ ]:
with open(Path.cwd() / 'Data' / 'Shakespeare.txt', 
          'r', encoding= 'UTF-8') as file_in:
    complete_works = file_in.readlines()
    
print(len(complete_works))

In [ ]:
for line in complete_works[:40]:
    print(line.strip())

# What can we do with this text?

First of all, remember that the computer is not an English major.  

However, the computer is able to complete long repetitive actions without error or lack of concentration.  So, whatever we can think involving counting, assigning, or comparing, we can do.

Even these simple analyses can be quite revealing.  For example, do you know that instructor evaluations tend to use different words to describe the teaching of men and women? Do you know that letters of recommendation also tend to use different terms for describing the skills and potential of men and women? Or that different ethnic groups are described using different terms?

Second, the computer is not an English major, but you may very well be one.  For this reason you can instruct the computer on what to do that could be of interest to an English major.

But before we do any of that, it is useful to have a little refresher on `string` parsing. 



# `String` parsing refresher

In [ ]:
with open(Path.cwd() / 'Data' / 'spoiler_alert.txt', 
          'r', encoding= 'UTF-8') as file_in:
    spoiler_alert = file_in.readlines()
    
print(len(spoiler_alert))

Let's iterate through the lines to see what the data looks like:

In [ ]:
print(spoiler_alert)
print()

for line in spoiler_alert:
    print(f"-- {line}")

## Removing white spaces at start and end of lines

When we print line by line there is a lot of empty lines. This is due to an invisible character: `\n` aka the *new line character*.

Usually, it is helpful to get rid of white space that helps humans read but serves no purpose for computers. As you might recall, we can easily strip a `string` of those things.

In [ ]:
for line in spoiler_alert:
    print(f"-- {line.strip()}")

Three things to notice.

First, the extra empty lines have disappeared.

Second, actual empty lines did not, and neither did white spaces within a line.

Third, we have not modified our strings


In [ ]:
spoiler_alert[3]

## Adding line numbers

Using `enumerate`, we can easily get line number when we print. 

This is particularly helpful when we are trying to **slice** the `list` of line in order to get to what we want. 

In [ ]:
for i, line in enumerate(spoiler_alert):
    print(f"{i:>5} -- {line.strip()}")

## Searching for specific text in a line

Many times, you are interested in pulling certain lines for the entire text.  Maybe they contain a keyword of interest to you... or maybe they are spoken by a certain character... 

In [ ]:
for i, line in enumerate(spoiler_alert):
    if 'the' in line:
        print(f"{i:>4} -- {line.strip()}")

Why aren't lines 1 ("The potent poison quite o'ercrows my spirit.") or 3 ("But I do prophesy th' election lights") printed?

For one, **capitalization matters!**

For another, we are not **picking up contractions**.  (I'm assuming th' is a contraction of the).

Let's address capitalization first.

In [ ]:
# Make everything lower case
#
for i, line in enumerate(spoiler_alert):
    if 'the' in line.lower():
        print(f"{i:>4} -- {line.strip()}")
        
print()

# Make everything upper case
#
for i, line in enumerate(spoiler_alert):
    if 'THE' in line.upper():
        print(f"{i:>4} -- {line.strip()}")

<br>

<br>

Is this the only way to find something in a line? Of course not! 


The `.find()` method even tells us the character position of the match.

Not only does `find` tell us whether the text appears in the line, but also exactly where in the line the text **it appears for the first time**  (indexing from 0). 

If it doesn't find our search query it will return -1. 


In [ ]:
test_string = 'the thesaurus'
first_index = test_string.find('the ')
print(first_index)
print(test_string[first_index+1 :].find('the '))


Let's make this code a little more general...

In [ ]:
pattern = 'the'

for i, line in enumerate(spoiler_alert):
    first_index = line.lower().find( pattern )
    if first_index != -1:
        print(f"{i:>4} -- {line.strip()}")

Sometimes, it is helpful to see the slice of a string near a certain pattern.

Choose a number of characters `step` to the left and to the right of the pattern and print the corresponding slice of a line.

In [ ]:
step = 5
pattern = 'the'
pat_length = len(pattern)

for i, line in enumerate(spoiler_alert):
    first_index = line.lower().find( pattern )
    if first_index != -1:
        i_start = max(0,first_index - step)
        i_end = first_index + pat_length + step
        print(f"{i:>4} -- ...{line[i_start: i_end]}...")

<br>

<br>

<br>

<br>



So, capitalization is taken care of. What about contractions?

Let's assume that **th'** is always a contraction of **the**.

In [ ]:
for i, line in enumerate(spoiler_alert):
    if 'the' in line.lower() or "th'" in line.lower():
        print(f"{i:>4} -- {line.strip()}")

## Exercise

Finding characters in lines can be used for many purposes.  One of them is to select **slices** of a `string`.

As an example, assume that we want to break the text into sentences.

Sentences will be ended by periods, exclamation marks or question marks.  So go ahead and break `spoiler_alert` into sentences.



In [ ]:
sentences = [] # To store all sentences identified

end_sentence = ['.', '!', '?'] # Possible ends of sentences
  
new_sentence = ''  # Start every new sentence with an empty string

# Be aware that a sentence may end in the middle of a line 
#
for i, line in enumerate(spoiler_alert):
    print(f"{i:>4} -- {line.strip()}")

    # Your code here
    
    
print(sentences)     

# `String` splitting refresher

Lines and sentences are useful ways of splitting text. However, in many situations we want to be looking at individual words or, more accurately, what we tell the computer constitutes a word!


In [ ]:
for i, line in enumerate(spoiler_alert):
    print(f"{i:>4} -- {line.strip().split()}")


Not bad, ah?

The default option for `.strip()` and `'split()` acts on `whitespace`, which we can `import` from the `string` package.  

As you can see, it does an excellent job of breaking a line into words.

You will notice, however, that some words have some punctuation attached at the end.

We will take care of it easily enough and, in one fell swoop, remove the pesky punctuation.  To accomplish this, we will use another `import` from the `string` package $-$ `punctuation`.

It comprises the characters: 

> ! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~



In [ ]:
all_the_words = []  # This is where we will store our list of words

for i, line in enumerate(spoiler_alert):
    line_words = line.strip().split()
    print(f"{i:>4} -- {line_words}")
    
    for word in line_words:
        print(f"\t{word.rstrip(punctuation)}")
        all_the_words.append(word.rstrip(punctuation))

print(len(all_the_words))
print(all_the_words)

<br>

<br>



Now imagine that we want to count how frequently each words appears (for those words that appear at least once, that is).  Then we must account for the fact that capitalization does not change the word.  That is,  *The* and *the* are the same word.

**You know how to take care of this, right?**


<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>



In [ ]:
all_the_words = [] 

for i, line in enumerate(spoiler_alert):
    line_words = line.strip().split()
    print(f"{i:>4} -- {line_words}")
    
    for word in line_words:
#         print(f"\t{word.rstrip(punctuation)}")
        all_the_words.append(word.rstrip(punctuation).lower())
        
Counter(all_the_words)

Wow! Othello is so self-centered. It is all about **I** with him.

Alright, we know you are dying. Get over it.

# Back to the Complete Works

Now that you are refreshed, we can get back to the `complete_works`

In [ ]:
for i, line in enumerate(complete_works[:30]):
    print(f"{i:>4} -- {line.strip()}")

<br>

<br>

<br>

<br>



Unfortunately, consistency is far from perfect in how the plays are formatted.  This means that our code to retrieve the different plays is somewhat clunky.

I first noticed that a prior to the start of a play, there is one of the copyright statements:

> <<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM
SHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS
PROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE
WITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE
DISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS
PERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED
COMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY
SERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>

I thus use the first line of that statement as an indicator pattern. **I use a Boolean variable to signal that I may be within a stretch of lines with a play inside.**
    
Next, I noticed that plays start with the year of publication (or first performance...).  This is a numeric string, giving me another pattern to match.
    
If, within a few lines after finding a year, I find a line all in upper case, then I know that this is a play title. **I use a Boolean variable to signal that I have now found a play.**
    
I assume I remain inside the play until I find the pattern *THE END*.  At this point, I switch my Boolean variables to `False`.
    
The output of this code is to create a dictionary storing the names of the plays, their year of publication, and their start and end lines within the file.

In [ ]:
elect_pattern = '<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM'

plays = {}
inside_flag = False
found_play = False

for i, line in enumerate(complete_works):
    if not inside_flag and line.strip() == elect_pattern:
        inside_flag = True
        print(i)
        continue
        
    if inside_flag and line.strip().isnumeric():
#         print(counter, i, complete_works[i].strip())
#         print(complete_works[i+2].strip())
#         print(i, inside_flag, found_play)
        if ( complete_works[i+2].upper() == complete_works[i+2] or
             complete_works[i+3].upper() == complete_works[i+3] or 
             complete_works[i+6].upper() == complete_works[i+6]):
            found_play = True
            year_play = int( line.strip() )
            
            if complete_works[i+2].strip() == '':
                if complete_works[i+3].strip() == '':
                    title_play = complete_works[i+6].strip()
                    plays[title_play] = [year_play, i+6]
                else:
                    title_play = complete_works[i+3].strip()
                    plays[title_play] = [year_play, i+3]
            else:    
                title_play = complete_works[i+2].strip()
                plays[title_play] = [year_play, i+2]
        continue
            
    if found_play and line.strip() == 'THE END':
        plays[title_play].append(i)
        print(title_play, plays[title_play])
        found_play = False
        inside_flag = False

        

In [ ]:
# And here we have it

plays

<br>

<br>

<br>

<br>




We will be focusing on Othello, or, more precisely *THE TRAGEDY OF OTHELLO, MOOR OF VENICE*

We have all the information to select this play in our `plays` dictionary.


In [ ]:
title = 'THE TRAGEDY OF OTHELLO, MOOR OF VENICE'
print(title)
print(plays[title])

print(f"\nThis play was published in {plays[title][0]}.\n")
line_start = plays[title][1]
line_end = plays[title][2]

the_play = complete_works[line_start:line_end+1]

print(f"It takes {len(the_play)} lines in the file.\n")
 

Before moving on, **always** make sure you really have what you _think_ you have. 

In [ ]:
print(the_play[:40])
print()

print(the_play[-40:])

## Getting the character names


Great! We have the play!

You will also notice that after **Dramatis Personae**, there is a list of characters in the play.

And it all ends with the copyright statement.

In [ ]:
start_pattern = 'Dramatis Personae'
end_pattern = elect_pattern

# Get lines with listing of characters
#
found_personae = False
for i, line in enumerate(the_play):
    if start_pattern in line:
        start_line = i + 1
        continue 
        
    if end_pattern in line:
        end_line = i
        break
        
personae_text = the_play[start_line: end_line]
print(personae_text)
            


<br>

<br>

<br>

<br>



**NOTICE THAT THE MANNER IN WHICH THIS INFORMATION IS FORMATED IS NOT IDENTICAL FOR ALL PLAYS.**

We want to get the characters' names and descriptions.  **What would be a good data structure for those data?**




In [ ]:
personae = {}  ## Yes, a dictionary!!!

for i, line in enumerate(personae_text):
    if line == '\n':
        continue

    print(f"{i:>3} -- {line.strip()}")
    
    # Split by comma, take first item if is all caps, 
    # join the rest
    
    items = line.strip().split(',')
#     print(items)
    
    if items[0].upper() == items[0]:
        name = items[0].strip()
        description = ','.join(items[1:]).strip()
        personae[name] = description
#         print(name, '--', description)

print()
personae


## Getting the lines from a specific character

Below, you can see the beginning of ACT I. 

You will notice a nice pattern to how lines are assigned to a character


In [ ]:
snippet = the_play_othello[40:53]

print(Fore.RED, f"   --12345678901234567890", Style.RESET_ALL)
for i, line in enumerate(snippet):
    print(f"{i:>4}--{line.rstrip()}")


The start of a character's line begins with the two spaces followed by the character's name capitalized and a period.

All the following lines that start with four spaces are assigned to that character. And then, the lines of a new character start.

```
"  CHARACTER. blahblahblah
     blahblahblah"
```

We can work with that, right?

Start by getting RODERIGO's lines from the `snippet`. 

In [ ]:
character = 'RODERIGO'
character_lines = []

found_lines = False
for i, line in enumerate(snippet):
    if line.split('.')[0] == '  ' + character:
        print(f"{i:>4}--{line.rstrip()}")
        found_lines = True
        character_lines.append(line)
        continue
    
    if found_lines:
        if line[:4] == '    ':
            print(f"{i:>4}--{line.rstrip()}")
            character_lines.append(line)
        else:
            found_lines = False
            continue


Go ahead and modify the code to read the lines from IAGO in the snippet.

# Getting and working a character's lines


Let's start by writing a function that reads and stores a character's lines given the lines in a play and the character's name.

In [ ]:
def read_character_lines( character, play_lines ):
    """
    This function takes the name of a character and the lines from the play
    extracted from GP's Complete Works of William Shakespeare and returns
    a list with all the lines from that character in the play
    
    inputs:
        character -- str
        play_lines -- list of str
        
    returns:
        character_lines -- list of str
    """
    character_lines = []
    character = character.upper()
    
    start_string = ' '*2 + character
    continuation_string = ' '*4
    
    found_lines = False
    for i, line in enumerate(play_lines):
        if line.split('.')[0] == start_string:
            found_lines = True
            character_lines.append(line.replace(start_string+ '.', '').strip())
            continue

        if found_lines:
            if line[:len(continuation_string)] == continuation_string:
                character_lines.append(line.strip())
            else:
                found_lines = False
                continue 
    
    return character_lines

In [ ]:
othello_lines = read_character_lines('othello', the_play)

print(len(othello_lines))
print(othello_lines[:10])

In [ ]:
iago_lines = read_character_lines('iago', the_play)
print(len(iago_lines))

In [ ]:
desdemona_lines = read_character_lines('desdemona', the_play)
print(len(desdemona_lines))

In [ ]:
brabantio_lines = read_character_lines('brabantio', the_play)
print(len(brabantio_lines))

# Bag of words approaches

Many NLP techniques make use of the so-called **bag of words** model. 

The idea is that you just get the words in some text and completely forget about their order and restrict yourselves to things that can be counted.

In order to move forward with this approach, it is helpful to write a function that given a list of lines breaks them into list of words.

That is your mission for the next few minutes, if you choose to take it.

In [ ]:
def extract_words_from_lines( character_lines ):
    """
    This function takes a list with all the lines from a character 
    in the play and returns a list of words
    
    inputs:
        character_lines -- list of str
        
    returns:
        character_words -- list of str
    """
    character_words = []
        
    return character_words

Ok, let's get the words spoken by Othello and use the `Counter` to get some statistics

In [ ]:
othello_words = extract_words_from_lines(othello_lines)

print(f"Othello's lines comprise {len(othello_words)} words.\n")
print(f"Othello's lines comprise {len(set(othello_words))} "
      f"unique words.\n")

othello_counter = Counter(othello_words)

print(len(othello_words)/ len(set(othello_words)))

**That means that each unique word is occurs an average of about 3.8 times.**

How good is this average a description of common words?

You can then get his most common words

In [ ]:
othello_counter.most_common(20)

**Interesting how the most common words appear about 50 times more frequently than one would expect**.

Some of those are clearly the usual suspects: *the*, *and*, *to*, *of*, *a*.

Not so typically, are *I* and *me*.

As we know, there is no **I** in **team**.

In [ ]:
print(othello_counter['we'])
print(othello_counter['team'])
print(othello_counter['othello'])

Since the most common words appear so frequently, the least common words must appear only a single time.

**Moreover, there should be many of those single occurrence words**.

In [ ]:
othello_counter.most_common()[-20:]


In [ ]:
othello_counter.most_common()[-1020:-1000]

Yes, over a thousand words appear only once.

This seems like a very strange distribution doesn't it?  There are a lot of words with very low probability of occurring and then a few with a high probability of occurring.

It is almost as if there was a country where most of the population was only 1 foot tall but then there where a couple 200 feet tall giants walking around.


## Zipf's law

Because this distribution is so strange, we will look at it in a little more detail.


In [ ]:
# Get frequency of words from Counter object
#
counts = list( dict(othello_counter).values() )

print(counts[:30])

In [ ]:
fig = plt.figure( figsize = (6, 4) )
ax = fig.add_subplot( 111 )
half_frame(ax, 'Number of occurrences', 'Frequency', font_size = my_fontsize)

ax.hist( counts, bins = np.arange(-0.5, 250.5, 10), 
         align = 'mid', rwidth = 0.9, label = 'Othello')

ax.legend(loc = 'best', frameon = False, fontsize = my_fontsize)

plt.tight_layout()

Not the most enlightening plot...

Setting a logarithmic scale on the y-axis will help 

In [ ]:
fig = plt.figure( figsize = (6, 4) )
ax = fig.add_subplot( 111 )
half_frame(ax, 'Number of occurrences', 'Frequency', font_size = my_fontsize)

ax.hist( counts, bins = np.arange(-0.5, 250.5, 10), 
         align = 'mid', rwidth = 0.9, label = 'Othello' )

ax.semilogy()
ax.legend(loc = 'best', frameon = False, fontsize = my_fontsize)

plt.tight_layout()

Better, but not yet great. Let's make the x-axis have a logarithmic scale too...

In [ ]:
fig = plt.figure( figsize = (6, 4) )
ax = fig.add_subplot( 111 )
half_frame(ax, 'Number of occurrences', 'Frequency', font_size = my_fontsize)

ax.hist( counts, bins = np.arange(0.5, 250.5, 10), 
         align = 'mid', rwidth = 0.9, label = 'Othello' )

ax.loglog()
ax.legend(loc = 'best', frameon = False, fontsize = my_fontsize)

plt.tight_layout()

This is not the best plot, but you can kind of see how the frequency decays as a straight like in this double **logarithmic plot**.

Such a pattern is indicative of a power-law decay

> $P(k) \propto k^{-\alpha}$

In the context of human languages this is called evidence for [Zipf's law](https://en.wikipedia.org/wiki/Zipf%27s_law). 

Because the support of the data is so large, the frequency plot gets to be quite noisy for large values of $k$.  

For this reason, it is typically best to plot the survival function instead of the frequency.

The survival function shows the number of values larger than $k$.

The next cell shows how this is calculated.


In [ ]:
k_values = []
survival_function = []

n = len(counts)
for k in sorted(counts):
    if k not in k_values:
        k_values.append(k)
        survival_function.append(n)
        n -= 1
    else:
        n -= 1
        
fig = plt.figure( figsize = (6, 4) )
ax = fig.add_subplot( 111 )
half_frame(ax, 'Number of occurrences', 'Survival function', font_size = my_fontsize)
ax.loglog()

# Add a line as a guide to the eye
ax.plot([1, 200], [1400, 3.8], 'b-', lw = 6, alpha = 0.3)

ax.plot( k_values, survival_function, 'r-', lw = 2, label = 'Othello' )

ax.legend(loc = 'best', frameon = False, fontsize = my_fontsize)

plt.tight_layout()   


## Text entropy

Bag of words approaches allow for easy calculation of many things. Above, we considered three quantities: number of words, size of vocabulary (aka, number of unique words), and distribution of word frequencies.

While is seems hard to compare distributions, the fact is that typical corpus generated from human language obey Zifp's law.  Moreover, the exponent in Zipf's law is related to measures of concentration. The smaller the value of the exponent $\alpha$ the more concentrated the speech is on a few words. Thus, the distribution also gives us a single number with which to compare characters.

Another way to measure someone's speech is by using **entropy**. [Entropy](https://en.wikipedia.org/wiki/Entropy) is a concept introduced in the study of heat. It measures disorder and in a sense tells us how hard to predict something is.

We are all much more predictable then we like to think. That is the reason why we are so easy to make fun of.  For instance, I seem to love the word *essentially*. Or at least that is what my so-called loved ones claim.

Entropy has also been introduced in the context of [Information Theory](https://en.wikipedia.org/wiki/Entropy_%28information_theory%29) where it is related to how surprising new information is.  When we listen to speech, every new word uttered brings new information. Sometimes, the amount of new information is close to zero because we already know what is coming

> baa baa black sheep
>
> have you any...


**So, how is entropy calculated?**

In the context of language, we can say that our speech is composed of words. Each word is an instance of a token (aka, unique word).  We can associate with each token a probability of selection. 

For example, let's say that our vocabulary has three tokens -- *hello*, *friends*, and *folks* -- and that each token has an associated probability of being selected. Then we can create a speech generator. Yes this is AI!





In [ ]:
vocabulary = {'hello': 0.5, 'friends': 0.35, 'folks': 0.15}

speech_size = 100
speech = []
for i in range(speech_size):
    x = random()
    p = 0
    for key in vocabulary:
        p += vocabulary[key]
        if x < p:
            speech.append(key)
            break
    
print(speech)
Counter(speech)

speech_counter = Counter(speech)

Entropy is expressed as a function of the probabilities of the tokens:


> $H(X) = -\sum ~ p_i ~\log p_i $

Where $i$ is the index for a token and $p_i$ is the probability of that token appearing in the text. 

In our context, the best estimate that we have of the probability of any given token is the ratio of the number of occurrences to the number of words.

> $p_i = \frac{n_i}{N}$

So, go ahead and write a function to calculate the entropy of a text given a `Counter` object.

In [ ]:
def text_entropy( counter ):
    """
    This function takes a Counter object and returns the entropy of the 
    underlying text.
    
    inputs:
        counter -- Counter object
        
    returns:
        entropy -- float
    """
    entropy = 0
    
    # Code here

    return entropy


In [ ]:
print(f"The entropy of the speech generator is "
      f"{text_entropy(speech_counter):.2f}\n")

print("The entropy of Othello's speech is "
      f"{text_entropy(othello_counter):.2f}\n")



You can see that the entropy scores for the these two forms of speech are quite different.

Not surprisingly, Othello's speech has much higher entropy -- that is, is much less predictable.

What about the speech of other the characters?


# Exercises

Go ahead and calculate the speech entropy of other major characters. 

So far we have calculated the properties of the entirety of the lines of a character.  However, it could be that the properties of the speech change in the course of the play.

How would you go about doing this?

Yes, exactly, calculate the change in properties of the speech of Othello over a set of 1000 consecutive words and plot how other speech properties change across the play.

In the previous exercises, you calculated a bunch of numbers.  Sometimes, those numbers appear to be quite different (1.46 *vs.* 8.84). Other times, not so much.

How would you be able to estimate the uncertainty in those number?

Is it $8.84 \pm 0.01$ or $8.8 \pm 1.0$?